# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 6 2022 12:00AM,238805,10,0085739772,ISDIN Corporation,Released
1,Jun 6 2022 12:00AM,238805,10,0085739773,ISDIN Corporation,Released
2,Jun 6 2022 12:00AM,238805,10,0085739827,ISDIN Corporation,Released
3,Jun 6 2022 12:00AM,238805,10,0085739851,ISDIN Corporation,Released
4,Jun 6 2022 12:00AM,238805,10,0085739852,ISDIN Corporation,Released
5,Jun 6 2022 12:00AM,238805,10,0085739855,ISDIN Corporation,Released
6,Jun 6 2022 12:00AM,238805,10,0085739882,ISDIN Corporation,Released
7,Jun 6 2022 12:00AM,238805,10,0085737384,ISDIN Corporation,Released
8,Jun 6 2022 12:00AM,238805,10,0085737392,ISDIN Corporation,Released
9,Jun 6 2022 12:00AM,238805,10,0085739932,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
33,238805,Released,36
34,238806,Released,1
35,238809,Released,1
36,238810,Released,7
37,238812,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
238805,NaN,NaN,36.0
238806,NaN,NaN,1.0
238809,NaN,NaN,1.0
238810,NaN,NaN,7.0
238812,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
238711,0.0,0.0,1.0
238720,0.0,0.0,1.0
238721,0.0,0.0,1.0
238748,0.0,0.0,1.0
238752,0.0,1.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
238711,0,0,1
238720,0,0,1
238721,0,0,1
238748,0,0,1
238752,0,1,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,238711,0,0,1
1,238720,0,0,1
2,238721,0,0,1
3,238748,0,0,1
4,238752,0,1,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,238711,,,1
1,238720,,,1
2,238721,,,1
3,238748,,,1
4,238752,,1,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 6 2022 12:00AM,238805,10,ISDIN Corporation
36,Jun 6 2022 12:00AM,238809,10,"Amcyte Pharma, Inc."
37,Jun 6 2022 12:00AM,238812,10,"Snap Medical Industries, LLC"
38,Jun 3 2022 4:07PM,238810,20,Alumier Labs Inc.
45,Jun 3 2022 3:30PM,238806,22,"NBTY Global, Inc."
46,Jun 3 2022 3:21PM,238803,10,Eywa Pharma Inc.
47,Jun 3 2022 3:21PM,238804,10,ISDIN Corporation
61,Jun 3 2022 3:05PM,238802,10,"Methapharm, Inc."
62,Jun 3 2022 3:04PM,238801,22,"NBTY Global, Inc."
63,Jun 3 2022 3:03PM,238800,21,"NBTY Global, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jun 6 2022 12:00AM,238805,10,ISDIN Corporation,,,36
1,Jun 6 2022 12:00AM,238809,10,"Amcyte Pharma, Inc.",,,1
2,Jun 6 2022 12:00AM,238812,10,"Snap Medical Industries, LLC",,,1
3,Jun 3 2022 4:07PM,238810,20,Alumier Labs Inc.,,,7
4,Jun 3 2022 3:30PM,238806,22,"NBTY Global, Inc.",,,1
5,Jun 3 2022 3:21PM,238803,10,Eywa Pharma Inc.,,,1
6,Jun 3 2022 3:21PM,238804,10,ISDIN Corporation,,,14
7,Jun 3 2022 3:05PM,238802,10,"Methapharm, Inc.",,,1
8,Jun 3 2022 3:04PM,238801,22,"NBTY Global, Inc.",,,1
9,Jun 3 2022 3:03PM,238800,21,"NBTY Global, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 6 2022 12:00AM,238805,10,ISDIN Corporation,36,,
1,Jun 6 2022 12:00AM,238809,10,"Amcyte Pharma, Inc.",1,,
2,Jun 6 2022 12:00AM,238812,10,"Snap Medical Industries, LLC",1,,
3,Jun 3 2022 4:07PM,238810,20,Alumier Labs Inc.,7,,
4,Jun 3 2022 3:30PM,238806,22,"NBTY Global, Inc.",1,,
5,Jun 3 2022 3:21PM,238803,10,Eywa Pharma Inc.,1,,
6,Jun 3 2022 3:21PM,238804,10,ISDIN Corporation,14,,
7,Jun 3 2022 3:05PM,238802,10,"Methapharm, Inc.",1,,
8,Jun 3 2022 3:04PM,238801,22,"NBTY Global, Inc.",1,,
9,Jun 3 2022 3:03PM,238800,21,"NBTY Global, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 6 2022 12:00AM,238805,10,ISDIN Corporation,36,,
1,Jun 6 2022 12:00AM,238809,10,"Amcyte Pharma, Inc.",1,,
2,Jun 6 2022 12:00AM,238812,10,"Snap Medical Industries, LLC",1,,
3,Jun 3 2022 4:07PM,238810,20,Alumier Labs Inc.,7,,
4,Jun 3 2022 3:30PM,238806,22,"NBTY Global, Inc.",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 6 2022 12:00AM,238805,10,ISDIN Corporation,36.0,NaN,NaN
1,Jun 6 2022 12:00AM,238809,10,"Amcyte Pharma, Inc.",1.0,NaN,NaN
2,Jun 6 2022 12:00AM,238812,10,"Snap Medical Industries, LLC",1.0,NaN,NaN
3,Jun 3 2022 4:07PM,238810,20,Alumier Labs Inc.,7.0,NaN,NaN
4,Jun 3 2022 3:30PM,238806,22,"NBTY Global, Inc.",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 6 2022 12:00AM,238805,10,ISDIN Corporation,36.0,0.0,0.0
1,Jun 6 2022 12:00AM,238809,10,"Amcyte Pharma, Inc.",1.0,0.0,0.0
2,Jun 6 2022 12:00AM,238812,10,"Snap Medical Industries, LLC",1.0,0.0,0.0
3,Jun 3 2022 4:07PM,238810,20,Alumier Labs Inc.,7.0,0.0,0.0
4,Jun 3 2022 3:30PM,238806,22,"NBTY Global, Inc.",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3104171,76.0,28.0,8.0
15,238755,1.0,0.0,0.0
16,716332,3.0,0.0,0.0
18,716236,3.0,0.0,0.0
19,477562,2.0,1.0,0.0
20,716359,19.0,21.0,18.0
21,477598,2.0,0.0,0.0
22,955166,4.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3104171,76.0,28.0,8.0
1,15,238755,1.0,0.0,0.0
2,16,716332,3.0,0.0,0.0
3,18,716236,3.0,0.0,0.0
4,19,477562,2.0,1.0,0.0
5,20,716359,19.0,21.0,18.0
6,21,477598,2.0,0.0,0.0
7,22,955166,4.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,76.0,28.0,8.0
1,15,1.0,0.0,0.0
2,16,3.0,0.0,0.0
3,18,3.0,0.0,0.0
4,19,2.0,1.0,0.0
5,20,19.0,21.0,18.0
6,21,2.0,0.0,0.0
7,22,4.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,76.0
1,15,Released,1.0
2,16,Released,3.0
3,18,Released,3.0
4,19,Released,2.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,18,19,20,21,22
Status,,,,,,,,
Completed,8.0,0.0,0.0,0.0,0.0,18.0,0.0,0.0
Executing,28.0,0.0,0.0,0.0,1.0,21.0,0.0,0.0
Released,76.0,1.0,3.0,3.0,2.0,19.0,2.0,4.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,18,19,20,21,22
0,Completed,8.0,0.0,0.0,0.0,0.0,18.0,0.0,0.0
1,Executing,28.0,0.0,0.0,0.0,1.0,21.0,0.0,0.0
2,Released,76.0,1.0,3.0,3.0,2.0,19.0,2.0,4.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,18,19,20,21,22
0,Completed,8.0,0.0,0.0,0.0,0.0,18.0,0.0,0.0
1,Executing,28.0,0.0,0.0,0.0,1.0,21.0,0.0,0.0
2,Released,76.0,1.0,3.0,3.0,2.0,19.0,2.0,4.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()